In [1]:
#import library
from libsbml import readSBML
from cobra import io
from cobra.core import Metabolite
from Functions import find_average

#import sbml file
model = io.sbml.create_cobra_model_from_sbml_file("/home/sanu/Documents/Scripts/git/plantcoremetabolism-model/PlantCoreMetabolism_v1_2_3.xml")

#change Biomass_tx to Protein_biomass
rxn = model.reactions.get_by_id("Biomass_tx")
rxn.id = "Protein_biomass_tx"
for met in ["K_b","Ca_b","Mg_b"]:
    met  = model.metabolites.get_by_id(met)
    coeff = rxn.metabolites.get(met)
    rxn.add_metabolites({met:-1*coeff})
met = Metabolite("Protein_b",name="Protein_biomass")
formula_dict = rxn.check_mass_balance()
met.formula = "".join([atom+str(formula_dict[atom]*-1) for atom in formula_dict.keys() if atom != "charge"])
met.charge = formula_dict["charge"]*-1
rxn.add_metabolites({met:1})

In [2]:
import pandas as pd
biomass = pd.DataFrame(data={"":["sSUCROSE_b","GLC_c","FRU_c","Starch_b","Cellulose_b","PALMITATE_c",
                                 "L_PHOSPHATIDATE_p","PHOSPHATIDYL_CHOLINE_r",
                                 "L_1_PHOSPHATIDYL_ETHANOLAMINE_r","DIACYLGLYCEROL_p","Protein_b","sMAL_b",
                                 "sCIT_b","sFUM_b","ARG_c","HIS_c","LYS_c","sASP_b",
                                 "sGLU_b","sSER_b","THR_c","ASN_c","sGLN_b","CYS_c",
                                 "GLY_c","PRO_c","sALA_b","VAL_c","ILE_c","LEU_c",
                                 "MET_c","PHE_c","TYR_c","TRP_c","sGABA_b"],
                             "leaf":[0.0]*35,"stem":[0.0]*35,"root":[0.0]*35,"seed":[0.0]*35,},dtype="float64")
biomass = biomass.set_index("")

In [3]:
proteinMW = model.metabolites.get_by_id("Protein_b").formula_weight

cobra/core/metabolite.py:103 UserWarning: 17.2020450391 is not an integer (in formula C17.2020450391H25.9492553613O5.7497158698N4.9536245179S0.1420323181)
cobra/core/metabolite.py:103 UserWarning: 25.9492553613 is not an integer (in formula C17.2020450391H25.9492553613O5.7497158698N4.9536245179S0.1420323181)
cobra/core/metabolite.py:103 UserWarning: 5.7497158698 is not an integer (in formula C17.2020450391H25.9492553613O5.7497158698N4.9536245179S0.1420323181)
cobra/core/metabolite.py:103 UserWarning: 4.9536245179 is not an integer (in formula C17.2020450391H25.9492553613O5.7497158698N4.9536245179S0.1420323181)
cobra/core/metabolite.py:103 UserWarning: 0.1420323181 is not an integer (in formula C17.2020450391H25.9492553613O5.7497158698N4.9536245179S0.1420323181)


### Mature leaf

Extract sucrose, starch and glucose diel content from Jeannette et al 2000, Figure 3

<img src="references/Screenshot_20190719_151346.png" height=50% width=50%>

In [13]:
df_matureleaf_data = pd.DataFrame(data={"day":[1,2,3,4],
                                        "sucrose dawn (g/m2)":[1.184615384615384,1.1846153846153795,
                                                               1.4923076923076835,1.5846153846153763],
                                        "sucrose dusk (g/m2)":[3.030769230769228,2.2923076923076877,
                                                               3.323076923076915,3.5538461538461448],
                                        "starch dawn (g/m2)":[0.9009900990099027,0.6831683168316811,
                                                              0.504950495049501,0.9009900990098938],
                                        "starch dusk (g/m2)":[1.8118811881188108,1.594059405940591,
                                                              1.3960396039603875,2.326732673267319],
                                        "glucose dawn (g/m2)":[0.3000000000000027,0.380645161290325,
                                                              0.30000000000000293,0.20322580645161725],
                                        "glucose dusk (g/m2)":[0.5548387096774199,0.3709677419354862,
                                                              0.661290322580645,0.3016129032258099]})
df_matureleaf_data.set_index("day",inplace=True,drop=True)
df_matureleaf_data["Δ sucrose(g/m2)"] = df_matureleaf_data["sucrose dusk (g/m2)"]- \
                                                df_matureleaf_data["sucrose dawn (g/m2)"]
df_matureleaf_data["Δ starch(g/m2)"] = df_matureleaf_data["starch dusk (g/m2)"]- \
                                                df_matureleaf_data["starch dawn (g/m2)"]
df_matureleaf_data["Δ glucose(g/m2)"] = df_matureleaf_data["glucose dusk (g/m2)"]- \
                                                df_matureleaf_data["glucose dawn (g/m2)"]
print("Starch:sucrose ratio=")
print(find_average(df_matureleaf_data["Δ starch(g/m2)"])/find_average(df_matureleaf_data["Δ sucrose(g/m2)"]))
df_matureleaf_data

Starch:sucrose ratio=
0.612778817745


,glucose dawn (g/m2),glucose dusk (g/m2),starch dawn (g/m2),starch dusk (g/m2),sucrose dawn (g/m2),sucrose dusk (g/m2),Δ sucrose(g/m2),Δ starch(g/m2),Δ glucose(g/m2)
day,,,,,,,,,
1,0.300000,0.554839,0.900990,1.811881,1.184615,3.030769,1.846154,0.910891,0.254839
2,0.380645,0.370968,0.683168,1.594059,1.184615,2.292308,1.107692,0.910891,-0.009677
3,0.300000,0.661290,0.504950,1.396040,1.492308,3.323077,1.830769,0.891089,0.361290
4,0.203226,0.301613,0.900990,2.326733,1.584615,3.553846,1.969231,1.425743,0.098387


### Leaf

Extract lipid  from Gu et al 2017, Figure 3

<img src="references/Screenshot_20190719_162426.png" height=50% width=50%>

In [16]:
df_leaf_content_1 = pd.DataFrame(data={"GDGD (%lipids)":[19.43217665615142],
                                       "MGDG (%lipids)":[45.78864353312303],
                                       "SQDG (%lipids)":[1.4984227129337468],
                                       "PG (%lipids)":[4.479495268138808],
                                       "PC (%lipids)":[5.047318611987379],
                                       "PE (%lipids)":[0.31545741324921295],
                                       "PI (%lipids)":[2.1608832807570963],
                                       "PA (%lipids)":[0.31545741324921295],
                                       "LPG (%lipids)":[0.3627760252365908]})
df_leaf_content_1

,GDGD (%lipids),LPG (%lipids),MGDG (%lipids),PA (%lipids),PC (%lipids),PE (%lipids),PG (%lipids),PI (%lipids),SQDG (%lipids)
0,19.432177,0.362776,45.788644,0.315457,5.047319,0.315457,4.479495,2.160883,1.498423


Extract biomass composition from Seaver 2015 maize leaf model

In [61]:
#import sbml file (had to remove 4-5 metabolite duplicates)
Sam_model = io.sbml.create_cobra_model_from_sbml_file("/home/sanu/Downloads/Seaver2015_Evidenced-Maize-Leaf-Model.sbml")
rxn = Sam_model.reactions.get_by_id("biomass0")

In [63]:
rxn.metabolites.get(Sam_model.metabolites.get_by_id("cpd19001_c0"))

-0.2469

In [52]:
df_leaf_content_2 = pd.DataFrame(data={"mets":["glucose","fructose","xylose","galactose","malate","citrate",
                                               "Arg","His","Lys","Asp","Glu","Ser","Thr","Asn","Gln","Cys",
                                               "Gly","Pro","Ala","Val","Ile","Leu","Met","Phe","Tyr","Trp"],
                                       "stoichiometry from Seaver et al 2015":
                                       [rxn.metabolites.get(Sam_model.metabolites.get_by_id("cpd19001_c0")),
                                        rxn.metabolites.get(Sam_model.metabolites.get_by_id("cpd30321_c0")),
                                        rxn.metabolites.get(Sam_model.metabolites.get_by_id("cpd01487_c0")),
                                        rxn.metabolites.get(Sam_model.metabolites.get_by_id("cpd00724_c0")),
                                        rxn.metabolites.get(Sam_model.metabolites.get_by_id("cpd00130_c0")),
                                        rxn.metabolites.get(Sam_model.metabolites.get_by_id("cpd00137_c0")),
                                        rxn.metabolites.get(Sam_model.metabolites.get_by_id("cpd00051_c0")),
                                        rxn.metabolites.get(Sam_model.metabolites.get_by_id("")),
                                        rxn.metabolites.get(Sam_model.metabolites.get_by_id("")),
                                        rxn.metabolites.get(Sam_model.metabolites.get_by_id("")),
                                        rxn.metabolites.get(Sam_model.metabolites.get_by_id("")),
                                        rxn.metabolites.get(Sam_model.metabolites.get_by_id("")),
                                        rxn.metabolites.get(Sam_model.metabolites.get_by_id("")),
                                        rxn.metabolites.get(Sam_model.metabolites.get_by_id("")),
                                        rxn.metabolites.get(Sam_model.metabolites.get_by_id("")),
                                        rxn.metabolites.get(Sam_model.metabolites.get_by_id("")),
                                        rxn.metabolites.get(Sam_model.metabolites.get_by_id("")),
                                        rxn.metabolites.get(Sam_model.metabolites.get_by_id("")),
                                        rxn.metabolites.get(Sam_model.metabolites.get_by_id("")),
                                        rxn.metabolites.get(Sam_model.metabolites.get_by_id("")),
                                        rxn.metabolites.get(Sam_model.metabolites.get_by_id("")),
                                        rxn.metabolites.get(Sam_model.metabolites.get_by_id("")),
                                        rxn.metabolites.get(Sam_model.metabolites.get_by_id("")),
                                        rxn.metabolites.get(Sam_model.metabolites.get_by_id("")),
                                        rxn.metabolites.get(Sam_model.metabolites.get_by_id("")),
                                        rxn.metabolites.get(Sam_model.metabolites.get_by_id(""))]})
df_leaf_content_2

,mets
0,sSUCROSE_b
1,GLC_c
2,FRU_c
3,Starch_b
4,Cellulose_b
5,PALMITATE_c
6,L_PHOSPHATIDATE_p
7,PHOSPHATIDYL_CHOLINE_r
8,L_1_PHOSPHATIDYL_ETHANOLAMINE_r
9,DIACYLGLYCEROL_p


In [57]:
temp = 0
for met in Sam_model.reactions.get_by_id("biomass0").reactants:
    print met.name
    if met.name == "ATP_c0" or met.name == "NADH_c0" or met.name == "H2O_c0":
        continue
    temp = temp + Sam_model.reactions.biomass0.metabolites.get(met)
    print temp

beta-D-Fructose_c0
-0.0363
Phytosphingosine_c0
-0.0364
Cl-_c0
-0.2464
L-Aspartate_c0
-0.2599
alpha-D-Glucose_c0
-0.5068
Citrate_c0
-0.5088
L-Tyrosine_c0
-0.5136
NADP_c0
-0.5137
NADPH_c0
-0.5138
sn-Glycerol_3-phosphate_c0
-0.5346
L-Leucine_c0
-0.5587
beta-D-Ribofuranose_c0
-0.5647
FMN_c0
-0.5648
GTP_c0
-0.56519
Pyridoxal_phosphate_c0
-0.56529
FAD_c0
-0.56539
L-Threonine_c0
-0.58049
alpha-L-Rhamnose_c0
-0.58129
5_10-Methylenetetrahydrofolate_c0
-0.58139
L-Valine_c0
-0.60169
L-Tryptophan_c0
-0.60369
ATP_c0
L-Glutamine_c0
-0.61239
dATP_c0
-0.61286
L-Asparagine_c0
-0.62066
cis-Aconitate_c0
-0.63466
Ubiquinone-9_c0
-0.63476
L-Phenylalanine_c0
-0.64276
Hexadecanoic_acid_c0
-0.64931
D-Galacturonate_c0
-0.66531
alpha-D-Mannose_c0
-0.67031
beta-Carotene_c0
-0.67071
dCTP_c0
-0.67115
L-Isoleucine_c0
-0.68285
CTP_c0
-0.68329
Biotin_c0
-0.68339
L-Glutamate_c0
-0.69939
L-Cysteine_c0
-0.70389
dGTP_c0
-0.70428
Choline_c0
-0.70438
R-Lipoic_acid_c0
-0.70448
alpha-L-Arabinose_c0
-0.74498
Xylose_c0
-0.9103

In [60]:
for met in Sam_model.reactions.get_by_id("biomass0").metabolites.keys():
    print met.id+"\t"+met.name[0:met.name.rindex("_")]+"\t"+str(Sam_model.reactions.get_by_id("biomass0").metabolites.get(met))
    #print met.id+"\t"+met.name[0:met.name.rindex("_")]+"\t"+str(100*Sam_model.reactions.get_by_id("biomass0").metabolites.get(met)/1.5)

#temp = pd.read_csv("/home/sanu/Downloads/cts-20190722154702.csv")
#temp

cpd30321_c0	beta-D-Fructose	-0.0363
cpd08926_c0	Phytosphingosine	-0.0001
cpd14958_c0	R-Lipoic_acid	-0.0001
cpd01698_c0	alpha-L-Arabinose	-0.0405
cpd01487_c0	Xylose	-0.1654
cpd00060_c0	L-Methionine	-0.0059
cpd00099_c0	Cl-	-0.21
cpd00041_c0	L-Aspartate	-0.0135
cpd19001_c0	alpha-D-Glucose	-0.2469
cpd00119_c0	L-Histidine	-0.0044
cpd00137_c0	Citrate	-0.002
cpd00724_c0	alpha-D-Galactose	-0.0346
cpd00069_c0	L-Tyrosine	-0.0048
cpd00006_c0	NADP	-0.0001
cpd00005_c0	NADPH	-0.0001
cpd00159_c0	S-Lactate	-0.006
cpd00098_c0	Choline	-0.0001
cpd03624_c0	Abscisate	-0.0001
cpd01234_c0	Campesterol	-0.00048
cpd03225_c0	Stigmasterol	-0.00064
cpd00008_c0	ADP	30.0
cpd00241_c0	dGTP	-0.00039
cpd00080_c0	sn-Glycerol_3-phosphate	-0.0208
cpd00302_c0	Zeatin	-0.0001
cpd16443_c0	beta-D-Ribofuranose	-0.006
cpd00130_c0	S-Malate	-0.006
cpd00121_c0	myo-Inositol	-0.003
cpd00345_c0	5-Methyltetrahydrofolate	-0.0001
cpd00059_c0	Ascorbate	-0.00096
cpd00062_c0	UTP	-0.00046
cpd01172_c0	Gibberellin_A3	-0.0001
cpd00156_c0	L-Valin

### Summary

- Since amino acid content was only available in g/gFW and FW/DW data was not available, diel starch and sucrose levels (in g/gFW) were used to predict baseline starch and sucrose levels in leaf


In [14]:
biomass.at["sSUCROSE_b","leaf"]=10*find_average(df_leaf_content_4["baseline sucrose (g/100gFW)"])/ \
                                                model.metabolites.get_by_id("SUCROSE_c").formula_weight
biomass.at["Starch_b","leaf"]=10*find_average(df_leaf_content_4["baseline starch (g/100gFW)"])/ \
                                                model.metabolites.get_by_id("SUCROSE_c").formula_weight
biomass.at["GLC_c","leaf"]=10*find_average(df_leaf_content_4["baseline simple sugars (g/100gFW)"])/ \
                                                model.metabolites.get_by_id("SUCROSE_c").formula_weight

biomass.at["ASN_c","leaf"]= find_average(df_leaf_content_3["Asn (nmol/50mgFW)"])/50
biomass.at["sGABA_b","leaf"]= find_average(df_leaf_content_3["GABA (nmol/50mgFW)"])/50
biomass.at["sGLN_b","leaf"]= find_average(df_leaf_content_3["Gln (nmol/50mgFW)"])/50
biomass.at["sGLU_b","leaf"]= find_average(df_leaf_content_3["Glt (nmol/50mgFW)"])/50
biomass.at["HIS_c","leaf"]= find_average(df_leaf_content_3["His (nmol/50mgFW)"])/50
biomass.at["PRO_c","leaf"]= find_average(df_leaf_content_3["Pro (nmol/50mgFW)"])/50
biomass.at["sSER_b","leaf"]= find_average(df_leaf_content_3["Ser (nmol/50mgFW)"])/50

### Stem



### Summary

- Sucrose, hexose content data is per FW and conversion while amino acid content data is per litre exudate. Conversions between these units is not available

Assumptions

### Root

### Summary

- sugars, amino acids and organic acids as %C and %N data could not be used directly.
- lipid composition in ug/cm2 cannot be translated to mg/gDW

### Seed


### References

- Jeannette, E., Reyss, A., Grégory, N., Gantet, P. and Prioul, J. ‐L. (2000) Carbohydrate metabolism in a heat‐girdled maize source leaf. Plant. Cell Environ., 23, 61–69. https://doi.org/10.1046/j.1365-3040.2000.00519.x.
- Saha, R., Suthers, P.F. and Maranas, C.D. (2011) Zea mays iRS1563: A Comprehensive Genome-Scale Metabolic Reconstruction of Maize Metabolism M. R. Andersen, ed. PLoS One, 6, e21784. https://doi.org/10.1371/journal.pone.0021784.
- Gu, Y., He, L., Zhao, C., Wang, F., Yan, B., Gao, Y., Li, Z., Yang, K. and Xu, J. (2017) Biochemical and Transcriptional Regulation of Membrane Lipid Metabolism in Maize Leaves under Low Temperature. Front. Plant Sci., 8, 1–13. Available at: https://doi.org/10.3389/fpls.2017.02053.